## Build model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

In [ ]:
def predict_proba(model, texts, tokenizer):
    
    encodings = [
        tokenizer.encode_plus(
            text, return_tensors="pt"
        ) for text in texts]
    
    results = [
        model(**encoding) for encoding in encodings
    ]
    
    list_results = [
        softmax(result[0].detach().numpy()[0]).tolist()
        for result in results
    ]
        
    return list_results

In [ ]:
predict_proba(model, ["good", "bad"], tokenizer)

# UNBOX

In [ ]:
import unboxapi
client = unboxapi.UnboxClient("ebcc2698-d638-11eb-938e-1e008a21a07a")
unboxapi.api.UNBOX_ENDPOINT = "http://0.0.0.0:8080/api" # Use this for local testing

# Package & Upload to Unbox

### Upload model

In [ ]:
from unboxapi.models import ModelType

model = client.add_model(
    function=predict_proba, 
    model=model,
    model_type=ModelType.transformers,
    class_names=['negative', 'positive'],
    name='transformers.sentiment_analyzer',
    description='this is my transformers sentiment model',
    tokenizer=tokenizer,
    requirements_txt_file='./requirements.txt'
)
model.to_dict()